In [ ]:
# Copyright 2023 Nokia
# Licensed under the BSD 3-Clause License.
# SPDX-License-Identifier: BSD-3-Clause

# Tested against SR OS 23.7.R1

import os
if os.environ.get("DEMO_PORT"):
    port = int(os.environ.get("DEMO_PORT"))
else:
    port = 830
if os.environ.get("DEMO_SSH_PORT"):
    ssh_port = int(os.environ.get("DEMO_SSH_PORT"))
else:
    ssh_port = 22
creds = {"hostname": os.environ.get("DEMO_HOST"), 
         "port": port, "ssh_port": ssh_port, 
         "username": os.environ.get("DEMO_USERNAME"), 
         "password": os.environ.get("DEMO_PASSWORD")} 

<img src="./_images/Nokia%20logo%20RGB_Bright%20blue.png" alt="NOKIA" width="400"/>

# SReXperts hackathon 2023 primer session

## pySROS

In 2021 Nokia introduced a Python 3 interpreter onto SR OS.  This Python 3 interpreter was released in conjunction with a Python library called `pySROS`.  

The Python library and the interpreter provide a fully model-driven interface to the routers ensuring a consistent interface whether executing on the device or remotely.

Over the years more features and functionality have been added to the solution which has found favor throughout our customer base for extending, personalizing and enabling additional functionality with SR OS.

## Installing and importing

### Download and install the library

The pySROS library is free to download and use and is automatically installed on all SR OS devices from release 21.7.R1.

In [ ]:
pip install --upgrade pysros

### Import the library into your project

The main part of the library is the `connect` method.  This must be imported.  Other methods may be imported as required.

In [ ]:
from pysros.management import connect

## Getting connected

### Connection object

Everything revolves around having a connection to the network device (on or off box). Obtaining a `Connection` object requires a few things if you are executing off-box:

* hostname/IP
* username
* password
* NETCONF port number (defaulted to 830)
* Whether the SSH hostkey should be verified (good practice in live networks and the default)

Each time a connection is made to a node, that node provides a list of YANG module names and versions that it implements.  This list is checked against pySROS' cached schema information (held in ~/.pysros).  If the exact set of modules and versions matches an existing cache entry then that cache is used.  If not the YANG modules are obtained from the router, compiled into a schema and cached. 

*Obtaining and compiling the YANG modules takes a small amount of time.  Using the cached information is much faster.*

In [ ]:
def get_connection():
    connection_object = connect(host=creds['hostname'], 
                            username=creds['username'], 
                            password=creds['password'], 
                            port=creds['port'], 
                            hostkey_verify=False)
    return connection_object

connection_object = get_connection()

When obtaining a `Connection` object on-box the parameters are not required, if they are provided they are simply ignored.  This means that your applications are portable between off-box and on-box as long as only the supported libraries are used.

This `Connection` object is what will be used to communicate with the routers model-driven API.

## Navigating and understanding the data

### Understanding and using paths to model-driven data structures

There are various path formats that you may be familiar with that allow you to reference a specific place in a YANG schema.  One such example is the **xpath** format.

The pySROS library requires the ability to provide paths, keys and key values to YANG modelled data elements in order to reference and manipulate them correctly.  The path format used is called the **json-instance-path**.

SR OS and pySROS are developed to be accessible to network engineers and developers alike by providing interfaces that simplify work for both.  The **json-instance-path** is one such example.

The quickest way to identify data that you would like to obtain is to navigate through the SR OS MD-CLI to the location of the desired data and type the `pwc json-instance-path` command.  This will provide the **json-instance-path** that you can cut and paste straight into Python.

For example:

```
[/]
A:admin@pe1# /state system time     

[/state system time]
A:admin@pe1# info
    zone-type system
    sntp {
        oper-state down
    }

[/state system time]
A:admin@pe1# pwc json-instance-path 
Present Working Context:
/nokia-state:state/system/time
```

As an example we will cut and paste the resulting path `/nokia-state:state/system/time` directly into a Python call to obtain that data.

In [ ]:
time_data = connection_object.running.get('/nokia-state:state/system/time')
print(time_data)

### Data structures

pySROS provides data in native Python formats making it very easy to manipulate as a developer (or as a network engineer).  The data that is obtained is augmented with all sorts of useful information from the YANG schema to enhance the model-driven management experience.

Each element of data is wrapped in a YANG class structure (`Leaf`, `LeafList`, `Action`, `Container`).  These wrapper classes also provide access to further information in the schema and can be used by other pySROS methods.

YANG lists are worth a specific mention.  YANG lists are defined as follows in YANG:

```
module example {
    namespace "urn:example";
    prefix "ex";
    revision "2023-06-10";
    container mycontainer {
        list mylist {
            key list-name;
            leaf list-name {
                type string;
            }
        }
    }
}
```

Which can be see in a `pyang` tree as:

```
module: example
  +--rw mycontainer
     +--rw mylist* [list-name]
        +--rw list-name    string
```

You will see that the key to the list is actually a child of the list itself.  If converted directly to a Python datastructure this would be very difficult to manipulate and so YANG list key values are made the key to Python `dict` data types.

Using the data obtained in `time_data` above we can show the wrapper classes at work by simply calling `printTree` on the data to output it in a tree structure.

In [ ]:
from pysros.pprint import printTree

printTree(time_data)

### The `Schema` class

The `Schema` class provides additional *read only* information from the compiled YANG schema for the device associated with the `Connection` object.  To see what options are available from the `Schema` class for a specific data element you can call `dir` on it.

In [ ]:
dir(time_data.schema)

You can see the elements not including the `_` character at the start and end are the YANG schema information available for that element (the top level container for `/nokia-state:state/system/time`).  In this example the available options are `module` and `namespace`.

In [ ]:
print("YANG module name is", time_data.schema.module)
print("YANG module namespace is", time_data.schema.namespace)

Exploring further we can look further into the data at the `zone-type` `Leaf`

In [ ]:
dir(time_data['zone-type'].schema)

Here you can see there is more data from the YANG:

* `mandatory`: Whether this leaf is mandatory to have in the data
* `module`: The name of the YANG module
* `namespace`: The namespace of the YANG module
* `yang_type`: The root YANG typedef for the element

In [ ]:
print("Mandatory:", time_data['zone-type'].schema.mandatory)
print("Module name:", time_data['zone-type'].schema.module)
print("Module namespace:", time_data['zone-type'].schema.namespace)
print("Root YANG typedef:", time_data['zone-type'].schema.yang_type)

## Handling larger datasets

### Lists and keys

The choice of how to obtain data and handle datasets depends on the specific use case and where the pySROS application is being executed (locally or remotely).  

When executing locally, memory should be a consideration but the speed to obtain data is unlikely to be as much of a factor as there is no transmission over a long distance.  Therefore, multiple `get` operations that collect smaller amount of data may be more efficient.

When executing remotely, memory is more bountiful but the transmission delay may be a factor.  In this instance, less `get` operations that return larger amounts of data may be more efficient.

There is no hard and fast rule for this though.

Let's get some data.

In [ ]:
big_data_1 = connection_object.running.get('/nokia-state:state/log/log-id')
print(big_data_1)

In [ ]:
printTree(big_data_1)

This data contains the information that I am looking for.  In this case, I would like to find the list of log IDs on the system.

I can query the data obtained to provide the information required:

In [ ]:
print(list(big_data_1.keys()))

However, I could obtain just the information I required directly from the node so I wouldn't have to obtain unrequired data which will consume memory.  In this case it is the list of keys of a YANG list so I can use the `get_list_keys` pySROS method.

In [ ]:
log_ids = connection_object.running.get_list_keys('/nokia-state:state/log/log-id')
print(log_ids)

You can see that, even on a relatively small dataset, there is an improvement in performance between selecting a larger dataset and whittling it down vs. obtaining a smaller dataset initially.

The improvement is calculated here:

In [ ]:
import time

big_set_start_time = time.perf_counter()
connection_object.running.get('/nokia-state:state/log/log-id')
big_set_duration = time.perf_counter() - big_set_start_time

small_set_start_time = time.perf_counter()
connection_object.running.get_list_keys('/nokia-state:state/log/log-id')
small_set_duration = time.perf_counter() - small_set_start_time

print("Obtaining list keys rather than the full dataset has a", 
      round((big_set_duration / small_set_duration)*100), 
      "percent improvement")


### Filters

We can also use filters to select specific items to match on or obtain from the device to further specify our desired dataset.

Let's query the YANG model offline to understand the makeup of this section of the YANG.

```
host % pyang -f tree -p .:ietf --tree-path=/state/log/log-id nokia-combined/nokia-state.yang
module: nokia-state
  +--ro state
     +--ro log
        +--ro log-id* [name]
           +--ro name          types-log:log-name
           +--ro oper-state?   types-sros:oper-state
           +--ro wrapped?      boolean
           +--ro next-event?   yang:counter64
           +--ro statistics
           |  +--ro logged-events?    yang:counter64
           |  +--ro dropped-events?   yang:counter64
           +--ro event* [sequence-number]
              +--ro sequence-number    uint64
              +--ro time?              yang:date-and-time
              +--ro severity?          types-log:severity-level
              +--ro application?       types-log:application-state
              +--ro event-id?          uint32
              +--ro event-name?        types-sros:named-item
              +--ro vrtr-name?         types-log:vrtr-name
              +--ro subject?           string
              +--ro message?           string
```

Let's obtain the key to the log-id entry `name` along with the `oper-state` leaf and the `statistics` container.

In [ ]:
logs_extended = connection_object.running.get('/nokia-state:state/log/log-id', filter={'name': {}, 'oper-state': {}, 'statistics': {}})
print(logs_extended)

In [ ]:
printTree(logs_extended)

There are two types of filters:

* Selection node
* Content match

A selection node filter is used in the example above and selects which fields you would like to obtain from the request.

A content match filter is used to select only data that exactly matches the specific string requested.

These two filter types may be used together.  Using the same example as above, we will extend it to match only entry 100 and we will continue to only retrieve specific fields.

In [ ]:
logs_extended = connection_object.running.get('/nokia-state:state/log/log-id', filter={'name': "100", 'oper-state': {}, 'statistics': {}})
print(logs_extended)

## Datastores

The eagle-eyed among you will have noticed that the word `running` appears in the statement to obtain data and list keys.  This is the NMDA datastore `running`.  The `running` and `candidate` datastores are supported.  The `candidate` datastore is a configuration datastore only and contains no state (YANG `config false`) information.

The examples so far have all been about obtaining state information but pySROS can of course be used for configuration as well.  

## Configuration

Configuration of a node is achieved by using the `set` method from the `Datastore` class.  The only datastore supported for making configuration changes is the `candidate` datastore as all interactions with SR OS use the model-driven transactional interfaces.  You can use the `running` datastore to obtain configuration information from the `running` datastore though.

In practice this means that a function call to configure the node will look like this:

```
connection_object.candidate.set(path, value)
```

`path` in the above call is the **json-instance-path** to the location in the YANG tree that you would like to configure from.  The `value` is the Python data structure containing the contents of your configuration.  This is formatted in the same way as the output seen in the examples above where we obtained state information.

In [ ]:
from pysros.wrappers import *

payload = {'PYSROS_PRIMER': Container({'name': Leaf('PYSROS_PRIMER'), 'default-action': Container({'action-type': Leaf('reject')})})}
connection_object.candidate.set('/nokia-conf:configure/policy-options/policy-statement', payload)

pySROS is written with a lazy in, strict out ethos.  That means that whilst, as above, you can provide very detailed and correctly Class-wrapped data, you can also provide much lazier input in the form of a basic Python dictionary (*dict*).

In [ ]:
payload = {'PYSROS_PRIMER': {'default-action': {'action-type': 'reject'}}}
connection_object.candidate.set('/nokia-conf:configure/policy-options/policy-statement', payload)

As the pySROS library is fully YANG schema aware, this lazy input option is also possible (***although not recommended***) for anywhere a **json-instance-path** is accepted.  For example: `/nokia-conf:configure/policy-options/policy-statement` may be entered as `/configure/policy-options/policy-statement`.

It is important to understand what actions the `set` method performs with the node:

1. Create a private candidate configuration (if one does not already exist for that `Connection` object)
2. Modify the private candidate configuration using a merge approach
3. Reconcile (update) the private candidate against the baseline (running) configuration
4. Validate the proposed configuration
5. Commit the proposed configuration

This behaviour has the effect that the `set` method immediately configures the node committing it to the running configuration.

This behaviour may be modified in a number of ways:

* The `method` parameter may be provided to the `set` method with the `replace` value.  This will perform any set overwriting any existing configuration (from the `path`)
* The `commit` parameter may be provided to the `set` method.  If this is done configuration is placed into the candidate configuration datastore but is not committed (applied) to the running configuration of the node.  This approach is useful if you would like to check the output prior to committing.

Now we will break apart the `set` method and perform the actions individually.  To do this we will introduce a few more methods: `lock`, `unlock`, `commit`, `compare` and `discard`.

First we will lock the device so no other session can interrupt our configuration actions.

In [ ]:
connection_object.candidate.lock()

Now let's configure the node without committing the configuration.

In [ ]:
connection_object.candidate.set('/nokia-conf:configure/log/log-id', {'name': '44', 'description': 'pySROS primer log'}, commit=False)

Using the `compare` method we can see what changes we will make to the node should we send the `commit` operation.

In [ ]:
print(connection_object.candidate.compare(output_format='md-cli'))

Let's assume that we had actually meant to configure log-id `45` not `44`.  We can discard the changes in the candidate configuration datastore using the `discard` method.

In [ ]:
connection_object.candidate.discard()
if not connection_object.candidate.compare(output_format='md-cli'):
    print("There are no candidate changes waiting for commit")

Now let's reconfigure the node using the correct `45` log-id.

In [ ]:
connection_object.candidate.set('/nokia-conf:configure/log/log-id', {'name': '45', 'description': 'pySROS primer log'}, commit=False)
print(connection_object.candidate.compare(output_format='md-cli'))

When happy we will commit the configuration to the running configuration datastore and unlock the device.

In [ ]:
connection_object.candidate.commit()
connection_object.candidate.unlock()

And we can now see our new log in the `running` configuration datastore.

In [ ]:
connection_object.running.get('/configure/log/log-id[name="45"]')

### Handling errors

Errors happen!  It's how you deal with them that counts.

Python allows developers to create errors by raising Exceptions which can then be handled.  A number of Exceptions are raised in pySROS for a variety of reasons.  Some of these are built in Python Exceptions and some are pySROS specific Exceptions.

The pySROS exceptions can be found [here](https://network.developer.nokia.com/static/sr/learn/pysros/latest/pysros.html#module-pysros.exceptions).

Python exceptions are handled by enclosing your activity in a `try`, `except` statement.

Let's look at an example.

The following returns an error.

In [ ]:
connection_object.running.get('/foo/bar')

You can see the output is messy.  This is very useful for developers but not as useful for users.  Towards the bottom you can see `InvalidPathError`.  This is the exception being raised in this particular case.  We can handle this neatly as follows:

In [ ]:
from pysros.exceptions import InvalidPathError

try:
    connection_object.running.get('/foo/bar')
except InvalidPathError as invalid_path_error:
    print("This didn't work because:", invalid_path_error)

Here is another example using the `SrosMgmtError`.

In [ ]:
from pysros.exceptions import SrosMgmtError

try:
    connection_object.candidate.unlock()
except SrosMgmtError:
    pass

for i in range(1,3):
    try:
        connection_object.candidate.lock()
    except SrosMgmtError as sros_mgmt_error:
        print(f"Attempt {i} didn't work because:", sros_mgmt_error)

try:
    connection_object.candidate.unlock()
except SrosMgmtError:
    pass

## Creating SR OS style outputs

pySROS provides some helper methods to allow SR OS "show" style outputs to be created.

The `Table` object will be used here to create a new "show" command that will reformat the output of BGP peers for a more personalized display.

In [ ]:
from pysros.pprint import Table

def obtain_bgp_data(connection_object):
    try:
        bgp_config_data = connection_object.running.get('/nokia-conf:configure/router[router-name="Base"]/bgp/neighbor')
        bgp_state_data = connection_object.running.get('/nokia-state:state/router[router-name="Base"]/bgp/neighbor')
        return bgp_config_data, bgp_state_data
    except LookupError as lookup_error:
        raise SystemExit(lookup_error)
    
def build_table(bgp_config_data, bgp_state_data):
    summary = "SReXperts 2023"
    columns = [(30, 'Peer'), (20, 'Group'), (20, 'State'), (30, 'Negotiated capabilities')]
    rows = []
    for neighbor in bgp_config_data:
        group = bgp_config_data[neighbor]['group'].data
        session_state = bgp_state_data[neighbor]['statistics']['session-state'].data
        try:
            negotiated_capability = bgp_state_data[neighbor]['statistics']['negotiated-family'].data
        except KeyError:
            negotiated_capability = None
        rows.append([neighbor, group, session_state, negotiated_capability])
    width = sum([col[0] for col in columns])
    table = Table("Summarized peerings", columns=columns,
                  showCount="Number of peers", summary=summary, width=width)
    return table, rows
        
    
bgp_config_data, bgp_state_data = obtain_bgp_data(connection_object)
table, rows = build_table(bgp_config_data, bgp_state_data)
table.print(rows)

## Execution of Python code

We have been using examples of pySROS applications running remotely so far.  Let's copy this application to the node and move over to the MD-CLI to show `pyexec` from a filename.

In [ ]:
import os
import paramiko

ssh = paramiko.SSHClient() 
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(creds['hostname'], username=creds['username'], password=creds['password'], port=creds['ssh_port'])
sftp = ssh.open_sftp()
sftp.put("./summarized_peering.py", "cf3:/summarized_peering.py")
sftp.close()
ssh.close()

Now let's configure a `python-script` on SR OS.

In [ ]:
connection_object.candidate.set('/nokia-conf:configure/python/python-script', {'srx': {'urls': ['cf3:/summarized_peering.py'],
                                                                                       'version': 'python3',
                                                                                       'admin-state': 'enable'}})


And we can see on the MD-CLI that we can now call `pyexec` directly using the configured `python-script` named `srx`.

Now we will use the MD-CLI command-alias feature to create our own show command embedded into the MD-CLI system.  We will create an alias called `srexperts_summary` that will appear in the `/show router bgp` section of the tree (only) that will call our configured pySROS application.


In [ ]:
payload = {'srexperts_summary': {'admin-state': 'enable',
                                 'description': 'Output summarized BGP sessions for SReXperts 2023',
                                 'python-script': 'srx',
                                 'mount-point': {'/show router bgp': {'path': '/show router bgp'}}}}
connection_object.candidate.set('/nokia-conf:configure/system/management-interface/cli/md-cli/environment/command-alias/alias', payload)

On the MD-CLI you will now see the command appear (after you log in again as command aliases are part of the user environment) in the `/show router bgp` tree with autocompletion and it's own context-sensitive help text (derived from the `description` field).

## Operations

We've talked about configuration and state information now, but pySROS can also instruct your SR OS node to perform operations using two methods:

1. YANG modelled `action` statements
2. MD-CLI commands

Whilst both are possible, they are in a numbered list above in order of usage recommendation.  YANG modelled `action` statements provide modelled data on input and output, whereas MD-CLI commands provide unstructured data and therefore (as we're talking about Python programming) should only be used where no action exists.

If you have a YANG modelled `action` available, always use that!

We will first use an MD-CLI command to dump the filter resource utilization on the iom.  To do this use the `cli` method on a `Connection` object.


In [ ]:
print(connection_object.cli('tools dump filter resources iom'))

Now we will use a YANG modelled `action` to validate our current license.

Let's look at the YANG model to determine the structure of the input for the `admin` function.

```
host % pyang -f tree -p .:ietf:nokia-combined --tree-path=/admin/system/license/validate nokia-oper-admin.yang
module: nokia-oper-admin
  +--ro admin
     +--ro system
        +--ro license
           +---x validate
              +---w input
              |  +---w file-url?   string
              +--ro output
                 +--ro operation-id?      types-operation:operation-id
                 +--ro start-time?        types-operation:operation-timestamp
                 +--ro results-path?      types-operation:operation-path
                 +--ro results
                 |  +--ro active-sros-version?                   types-sros:description
                 |  +--ro license-file?                          types-sros:url
                 |  +--ro license-name?                          types-sros:description
                 |  +--ro license-uuid?                          types-sros:description
                 |  +--ro license-secondary-uuid?                types-sros:description
                 |  +--ro machine-uuid?                          types-sros:description
                 |  +--ro license-description?                   types-sros:description
                 |  +--ro license-product?                       types-sros:description
                 |  +--ro license-applicable-versions?           types-sros:description
                 |  +--ro license-issue-time?                    yang:date-and-time
                 |  +--ro license-start-time?                    yang:date-and-time
                 |  +--ro license-end-time?                      yang:date-and-time
                 |  +--ro license-validation-status?             enumeration
                 |  +--ro validation-error-additional-details?   types-sros:very-long-description-or-empty
                 +--ro status?            types-operation:operation-status
                 +--ro error-message*     types-operation:operation-message
                 +--ro warning-message*   types-operation:operation-message
                 +--ro info-message*      types-operation:operation-message
                 +--ro end-time?          types-operation:operation-timestamp
```

Here we can see an `action` indicated by the `x` of `/nokia-oper-admin:admin/system/license/validate` and we can use the `action` method to execute this.

In [ ]:
license = connection_object.action('/admin/system/license/validate')
print(license)
printTree(license)

## Conclusion

This primer has given you a working introduction to the pySROS libraries from Nokia that present a clean Python API to Nokia nodes.  

pySROS has many more functions and features not explained here. For more information please refer to the documentation available on the [Nokia developer portal](https://network.developer.nokia.com) and the examples directory of the pySROS repository on GitHub:

* [pySROS documentation](https://network.developer.nokia.com/static/sr/learn/pysros/latest)
* [pySROS on GitHub](http://github.com/nokia/pysros)

If you have any further questions and would like more examples, assistance, presentations or tutorials please reach out to a Nokia representative.

All that remains now is to run the cells below to cleanup the devices you configured during this primer.

## Cleanup

This section simply cleans up configuration added to your node during this primer.

In [ ]:
def remove_files():
    import os
    import paramiko
    
    ssh = paramiko.SSHClient() 
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(creds['hostname'], username=creds['username'], password=creds['password'], port=creds['ssh_port'])
    sftp = ssh.open_sftp()
    try:
        sftp.remove("cf3:/summarized_peering.py")
    except FileNotFoundError as file_not_found:
        print("File not present, proceeding with cleanup:", file_not_found)
        pass
    sftp.close()
    ssh.close()

def unconfigure():
    paths = ['/nokia-conf:configure/log/log-id[name="45"]', '/nokia-conf:configure/policy-options/policy-statement[name="PYSROS_PRIMER"]',
             '/nokia-conf:configure/system/management-interface/cli/md-cli/environment/command-alias/alias[alias-name="srexperts_summary"]',
             '/nokia-conf:configure/python/python-script[name="srx"]']
    for path in paths:
        try:
            connection_object.candidate.delete(path)
            print(path, "deleted")
        except Exception as error:
            print(path, "not deleted:", error)
            pass
    
def cleanup():
    remove_files()
    unconfigure()

cleanup()